In [1]:
import folium
import pandas as pd
import numpy as np
import os
import json

In [2]:
europe_data_path =  os.path.join('data', 'tepsr_wc170.tsv')
europe_data_original = pd.DataFrame.from_csv(europe_data_path, sep='\t')


europe_data = europe_data_original.copy()

# Keeps only digits and dots from a string
def clean_number(n):
    return ''.join(c for c in str(n) if c.isdigit() or c == '.')

# Keep only rows for age group 20-64 and drop other age ranges
europe_data = europe_data.filter(regex='^PC,T,Y20-64,TOTAL', axis=0)

# Clean the values, as some contain letters
europe_data = europe_data.applymap(lambda x: clean_number(x))

# Rename the indexes to keep only the country code
europe_data.index = europe_data.index.to_series().str.replace(r'PC,T,Y20-64,TOTAL,(\w+)', r'\1')
europe_data.index.name = 'country_code'

# Keep only the 2016 column, which is the most recent data
europe_data = europe_data[['2016 ']]

# Useless below?
europe_data = europe_data.fillna('0.0').astype('float')

europe_data = europe_data.head(1)

europe_data.head()

,2016
country_code,
AT,5.8


In [3]:
#europe_topojson_path = os.path.join('topojson', 'europe.topojson.json')
europe_topojson_path = os.path.join('topojson', 'europe.geojson')

europe_topojson = json.load(open(europe_topojson_path))
europe_map = folium.Map(
    location=[53.5775, 23.106111],
    tiles='cartodbpositron',
    #tiles='Mapbox Bright',
    zoom_start=4
)

folium.GeoJson(europe_topojson).add_to(europe_map)

europe_map.choropleth(geo_data=europe_topojson, data=europe_data,
                   columns=['2016 '],
                   key_on='features.properties.FIPS',
                   fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                   legend_name='Unemployment rate')

europe_map

IndexError: list index out of range

Getting the Switzerland unemployement data from the provided website was a bit tricky. We first had to specify the encoding to utf-16. And also manually edit the csv file to remove the "month" and "total" element which appears to be a second layer of header which we don't need. We also rename the canton to match the json file. Il could be automaticly done with a maping, or with adding a new column with canton id.

In [38]:
from branca.colormap import linear

#Set the path and load the Json data.
swiss_topojson_path = os.path.join('topojson', 'ch-cantons.topojson.json')

#Load the map with a correct zoom on Switzerland.
swiss_map = folium.Map(location = [46.8, 8.33], tiles='Mapbox Bright', zoom_start = 7)

#Load the Switzerland unemployement's data.
swiss_unemployement_Sept2017 = os.path.join('data', 'swiss_unemployement_2.csv')
swiss_unemployement = pd.read_csv(swiss_unemployement_Sept2017, sep=';', encoding="utf-16")
swiss_unemployement = swiss_unemployement.drop('Mesures', 1).drop('Taux de chômage.1', 1).drop('Chômeurs inscrits.1', 1).drop("Demandeurs d'emploi.1", 1).drop("Demandeurs d'emploi non chômeurs.1", 1)

#Create a dictionnary in order to map a "canton" to its unemployement rate .
swiss_unemp_dict = swiss_unemployement.set_index("Canton")["Taux de chômage"]

#Create a colormap
colormap = linear.YlGn.scale(0, swiss_unemployement["Taux de chômage"].max())

#Create a dictionnary of color based on our "canton" - unemployement rate dictionnary.
color_dict = {key: colormap(swiss_unemp_dict[key]) for key in swiss_unemp_dict.keys()}

#The color function to apply on our "canton".
style_function = lambda x: {'fillColor': color_dict[x['properties']['name']],
                            'fillOpacity': 0.9,
                           }

folium.TopoJson(open(swiss_topojson_path, encoding="utf-8"),
                'objects.cantons',
                name='topojson',
                style_function=style_function
               ).add_to(swiss_map)

colormap.caption = 'Unemployment color scale in percent [%]'
colormap.add_to(swiss_map)

#save the map as an html element to show it on GitHub.
swiss_map.save(os.path.join('results', 'GeoJSON_SwissMap_1.html'))

swiss_map